<a href="https://colab.research.google.com/github/leonammeta8154/Imers-o_IA_Alura_Aula_05---Agentes.ipynb/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
%pip -q install google-genai

In [77]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [76]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [75]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar a página da Alura:** Fique de olho na página principal da Alura ([https://www.alura.com.br/](https://www.alura.com.br/)) e nas redes sociais deles (LinkedIn, Instagram, etc.). Eles costumam anunciar novos cursos e imersões por lá.
*   **Assinar a newsletter da Alura:** Assim, você receberá informações sobre os próximos lançamentos e eventos diretamente no seu e-mail.
*   **Verificar o fórum da Alura:** Às vezes, os próprios alunos ou instrutores compartilham informações sobre futuras edições no fórum da Alura.

Boa sorte!

In [74]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu de 12 a 16 de maio de 2025. As inscrições para a imersão foram até o dia 11 de maio de 2025. A Imersão IA é um evento 100% online e gratuito.

In [73]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: youtube.com, youtube.com, alura.com.br, youtube.com



In [72]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [71]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService # memoria interna desse agente.
from google.adk.tools import google_search # ferramenta do google
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [70]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [85]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [84]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name ="agente_buscador",
      model="gemini-2.0-flash",
      description="agente de buscar informações no google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é posspivel que ele não seja tão relevante aasim
      esse lançamenbto relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  # Executa o agente
  lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos_buscados

In [83]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""

        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lanlamentos mais recentes e revelantes buscador, você deve:
        usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os asuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [82]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [81]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [86]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: basquete
Maravilha! Vamos então criar o post sobre novidades em basquete

--- 📝 Resultado do Agente 1 (Buscador) ---



> Olá! Para fornecer as últimas notícias sobre os lançamentos mais relevantes no mundo do basquete, dentro do último mês, preparei algumas pesquisas no Google. Vamos ver o que está acontecendo!
> 
> 
> Com certeza! Aqui estão alguns dos lançamentos e notícias mais relevantes do mundo do basquete no último mês:
> 
> 1.  **Liga de Basquete Feminino (LBF) 2025:** A temporada de 2025 da LBF foi lançada oficialmente e começou em 8 de março, com um número recorde de 11 clubes participantes. A competição celebra 15 anos de fundação e traz novos projetos, com o objetivo de fortalecer a modalidade. A LBF e a Confederação Brasileira de Basketball (CBB) estão trabalhando juntas em novas medidas para a Liga em 2025, incluindo a participação de todos os times da LBF CAIXA 2025 em um Campeonato Brasileiro Interclubes (CBI) de base organizado pela CBB.
> 2.  **Parceria CBB e Winner Sports:** A Confederação Brasileira de Basketball fechou uma parceria com a Winner Sports, que agora é a tabela oficial do basquete brasileiro.
> 3.  **WNBA 2025:** A temporada da WNBA 2025 começou em 16 de maio e vai até meados de outubro. A liga conta com duas atletas brasileiras: Kamilla Cardoso, do Chicago Sky, e Damiris Dantas, do Indiana Fever.
> 4.  **Playoffs da NBA:** Os playoffs da NBA estão em andamento, com destaque para a classificação do Indiana Pacers para a final do Leste. Stephen Curry foi liberado para treinos leves, mas segue fora do jogo 5. Os horários e onde assistir aos jogos do fim de semana também estão sendo amplamente divulgados.
> 5.  **Liga Ouro 2025:** A Liga Ouro 2025 definiu os confrontos, datas, horários e transmissões das semifinais.
> 
> Além disso, vale mencionar que a ESPN preparou uma programação especial para o mês de maio, com grandes eventos de basquete, tênis e futebol.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay! Com base nas informações que você me forneceu, vou criar um plano para posts sobre os temas mais relevantes do basquete no último mês.
> 
> 
> Com base nas minhas pesquisas, o tema mais relevante para um post no momento é a **NBA Playoffs 2025**, devido ao seu apelo global e à fase decisiva em que se encontra.
> 
> **Tema:** NBA Playoffs 2025
> 
> **Pontos Relevantes:**
> 
> *   **Destaque para as Finais de Conferência:** Times classificados, confrontos, datas e horários dos jogos.
> *   **Desempenho dos Jogadores:** Análise do desempenho de jogadores chave e possíveis zebras.
> *   **Lesões:** Impacto de lesões em jogadores importantes e como isso afeta as chances de suas equipes.
> *   **Histórias:** Narrativas interessantes como times buscando quebrar longos jejuns ou rivalidades acirradas.
> *   **Transmissão:** Onde assistir aos jogos no Brasil.
> 
> **Plano de Assuntos para o Post:**
> 
> 1.  **Título:** NBA Playoffs 2025: Guia Completo das Finais de Conferência
> 2.  **Introdução:**
>     *   Breve resumo do que aconteceu nas rodadas anteriores dos playoffs.
>     *   Destaque para os times que chegaram às finais de conferência e suas jornadas.
> 3.  **Finais da Conferência Leste:**
>     *   Confronto: Indiana Pacers vs. Celtics/Knicks.
>     *   Análise das equipes: Pontos fortes, fracos e jogadores chave.
>     *   Impacto da lesão de Jayson Tatum, caso os Celtics avancem.
>     *   O que esperar da série: Previsões e fatores a serem observados.
> 4.  **Finais da Conferência Oeste:**
>     *   Confronto: Timberwolves vs. Thunder/Nuggets.
>     *   Análise das equipes: Pontos fortes, fracos e jogadores chave.
>     *   Duelo entre Nikola Jokić (Nuggets) e a defesa dos Timberwolves, caso se enfrentem.
>     *   O que esperar da série: Previsões e fatores a serem observados.
> 5.  **Brasileiros na NBA:**
>     *   Menção honrosa aos jogadores brasileiros que atuam na NBA e como eles se encaixam em seus respectivos times (se aplicável).
> 6.  **Onde Assistir:**
>     *   Informações sobre os canais de TV e plataformas de streaming que transmitem os jogos no Brasil.
> 7.  **Curiosidades e Narrativas:**
>     *   Estatísticas interessantes e recordes que podem ser quebrados.
>     *   Histórias inspiradoras de jogadores ou times superando adversidades.
> 8.  **Conclusão:**
>     *   Reafirmação da importância das finais de conferência.
>     *   Incentivo para os fãs acompanharem os jogos e torcerem por seus times favoritos.
> 9.  **Call to Action:**
>     *   Perguntar aos seguidores quem eles acham que vencerá as finais de conferência.
>     *   Incentivar o compartilhamento do post com outros fãs de basquete.
> 
> Este plano fornece uma estrutura abrangente para um post envolvente e informativo sobre a NBA Playoffs 2025.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🏀🔥 Finais de Conferência da NBA 2025: O Guia Que Você Precisa! 🔥🏀
> 
> A temporada tá pegando fogo e as Finais de Conferência da NBA estão imperdíveis! 🔥 Depois de rodadas eletrizantes, Indiana Pacers, Celtics/Knicks, Timberwolves e Thunder/Nuggets vão com tudo em busca do título.
> 
> 🔥 No Leste, Pacers x Celtics/Knicks prometem duelos INSANOS! Será que o talento dos Pacers supera a tradição dos Celtics ou a garra dos Knicks? 👀 E no Oeste, Timberwolves x Thunder/Nuggets é um choque de estilos que vai te deixar sem fôlego! Defesa implacável contra ataque explosivo. Qual será a receita para o sucesso? 🤯
> 
> 🇧🇷 E os nossos brasileiros na NBA? 🇧🇷 Eles também estão brilhando e mostrando o talento verde e amarelo nas quadras! ✨
> 
> 📺 Quer saber onde assistir? 📺 Se liga: os jogos serão transmitidos pelos canais [ESPN] e [Star+]. Prepare a pipoca e convoque a galera! 🍿
> 
> 🤔 E aí, quem leva essa? 🤔 Deixe seu palpite nos comentários e marque aquele amigo(a) que não perde um lance! 👇
> 
> #NBAPlayoffs #NBA2025 #BasqueteNaVeia #Alura 
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
